In [1]:
import pandas as pd
import json

In [297]:
medals = pd.read_csv('medals_per_year_country_sport_sex.csv')
sports = pd.read_csv('../data/sports_details.csv')

In [298]:
medals

,Year,Season,NOC,Sport,Sex,Medal,Event
0,1896,Summer,GBR,Weightlifting,M,Gold,"[""Weightlifting Men's Unlimited; One Hand""]"
1,1896,Summer,GBR,Weightlifting,M,Silver,"[""Weightlifting Men's Unlimited; Two Hands""]"
2,1896,Summer,GER,Athletics,M,Silver,"[""Athletics Men's 100 metres""]"
3,1896,Summer,GER,Gymnastics,M,Bronze,"[""Gymnastics Men's Rope Climbing""]"
4,1896,Summer,GER,Gymnastics,M,Gold,"[""Gymnastics Men's Horizontal Bar; Teams"" ""Gym..."
...,...,...,...,...,...,...,...
10371,2016,Summer,VEN,Athletics,F,Silver,"[""Athletics Women's Triple Jump""]"
10372,2016,Summer,VEN,Boxing,M,Bronze,"[""Boxing Men's Flyweight""]"
10373,2016,Summer,VEN,Cycling,F,Bronze,"[""Cycling Women's BMX""]"
10374,2016,Summer,VIE,Shooting,M,Gold,"[""Shooting Men's Air Pistol; 10 metres""]"


In [299]:
sports = sports[['Games','Sport','Sex','Name','Event','NOC']]

In [300]:
#sports['Season'] = sports.Games.apply(lambda x: x.split()[-1])

In [301]:
#sports['Year'] = sports.Games.apply(lambda x: x.split()[0])

In [302]:
sports

,Games,Sport,Sex,Name,Event,NOC
0,1896 Summer,Athletics,M,63,12,9
1,1896 Summer,Cycling,M,19,6,5
2,1896 Summer,Fencing,M,15,3,4
3,1896 Summer,Gymnastics,M,28,8,8
4,1896 Summer,Shooting,M,38,5,7
...,...,...,...,...,...,...
1391,2016 Summer,Water Polo,M,154,1,12
1392,2016 Summer,Weightlifting,F,103,7,62
1393,2016 Summer,Weightlifting,M,152,8,80
1394,2016 Summer,Wrestling,F,112,6,44


In [303]:
#sports_year_overviews = sports[['Sport','Sex','Event','Season','Year']]
sports_year_overviews = sports[['Sport','Games']].drop_duplicates()

In [304]:
#sports_year_overviews['Year'] = A

In [305]:
sports_order = sports_year_overviews[['Sport','Games']].groupby(['Sport'])['Games'].apply(lambda x: min(list(x)))
sports_year_dict = sports_order.apply(lambda x : int(x.split()[0])).to_dict()

In [306]:
sports_per_year = sports_year_overviews.groupby('Games')['Sport'].apply(lambda x: list(x))

In [307]:
#pd.DataFrame(sports_per_year.apply(lambda x:sorted(x, key=lambda y:sports_year_dict[y]))).to_json('sports_per_game.json')
#pd.DataFrame(sports_per_year.apply(lambda x:sorted(x, key=lambda y:sports_year_dict[y]))).to_csv('sports_per_game.csv')

In [308]:

participants = pd.read_csv('../data/athlete_events.csv')
participants = participants[['Name','Sex','Team','NOC','Games','Sport','Event','Medal']].rename(columns={'Sport':'Sport','Games':'Game'})


In [309]:
# Check if some games are dropped

sports_per_year.index.shape ==  participants.Game.unique().shape

True

In [310]:
( participants[['Sport','Game']].drop_duplicates().sort_values(by='Sport').reset_index(drop=True)['Sport'] == \
 sports_year_overviews.sort_values(by='Sport').reset_index(drop=True)['Sport']).any()

True

In [311]:
participants

,Name,Sex,Team,NOC,Game,Sport,Event,Medal
0,A Dijiang,M,China,CHN,1992 Summer,Basketball,Basketball Men's Basketball,NaN
1,A Lamusi,M,China,CHN,2012 Summer,Judo,Judo Men's Extra-Lightweight,NaN
2,Gunnar Nielsen Aaby,M,Denmark,DEN,1920 Summer,Football,Football Men's Football,NaN
3,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900 Summer,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...
271111,Andrzej ya,M,Poland-1,POL,1976 Winter,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,Piotr ya,M,Poland,POL,2014 Winter,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,Piotr ya,M,Poland,POL,2014 Winter,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,Bobsleigh,Bobsleigh Men's Four,NaN


In [312]:
participants.loc[participants.Event.apply(lambda x: 'Mixed' in x), 'Sex'] = 'Mixed'

In [313]:
participants['Event'] = participants.apply(lambda x: x.Event[len(x.Sport)+1:], axis=1)

In [314]:
import numpy as np
medals_translate = {'Gold':3, 'Silver':2,'Bronze':1}
participants['Medal'] = participants['Medal'].dropna().apply(lambda x: medals_translate[x])

In [315]:
participants['Medalist'] = participants[['Name','Medal']].dropna().apply(lambda x: list([x.Medal, x.Name]), axis=1)

In [316]:
participants

,Name,Sex,Team,NOC,Game,Sport,Event,Medal,Medalist
0,A Dijiang,M,China,CHN,1992 Summer,Basketball,Men's Basketball,NaN,NaN
1,A Lamusi,M,China,CHN,2012 Summer,Judo,Men's Extra-Lightweight,NaN,NaN
2,Gunnar Nielsen Aaby,M,Denmark,DEN,1920 Summer,Football,Men's Football,NaN,NaN
3,Edgar Lindenau Aabye,M,Denmark/Sweden,DEN,1900 Summer,Tug-Of-War,Men's Tug-Of-War,3.0,"[3.0, Edgar Lindenau Aabye]"
4,Christine Jacoba Aaftink,F,Netherlands,NED,1988 Winter,Speed Skating,Women's 500 metres,NaN,NaN
...,...,...,...,...,...,...,...,...,...
271111,Andrzej ya,Mixed,Poland-1,POL,1976 Winter,Luge,Mixed (Men)'s Doubles,NaN,NaN
271112,Piotr ya,M,Poland,POL,2014 Winter,Ski Jumping,"Men's Large Hill, Individual",NaN,NaN
271113,Piotr ya,M,Poland,POL,2014 Winter,Ski Jumping,"Men's Large Hill, Team",NaN,NaN
271114,Tomasz Ireneusz ya,M,Poland,POL,1998 Winter,Bobsleigh,Men's Four,NaN,NaN


In [317]:
import operator
from collections import Counter


def sum_MVP(_list):
    result = {}
    for pair in _list:
        if pair[1] in result:
            result[pair[1]] += pair[0]
        else:
            result[pair[1]] = pair[0]
    return sorted(result.items(),key=operator.itemgetter(1),reverse=True)[0:3]

# tie handling???
# we can list the medal details if we store the info a bit differently
# how to handle group events????? 



#grouped = pd.DataFrame(participants[['Game','Sport','Event']].drop_duplicates().groupby(['Game','Sport'], as_index=False)['Event'].apply(lambda x: list(x)))#.to_json()
grouped = pd.DataFrame(participants.groupby(['Game','Sport','Event'], as_index=False)['Team'].apply(lambda x: Counter(x)))#.to_json()
grouped['MVP'] = participants[participants['Medal'].notna()].groupby(['Game','Sport','Event'], as_index=False)['Medalist'].apply(lambda x:sum_MVP(list(x)))

grouped_general = pd.DataFrame(participants.groupby(['Game','Sport'], as_index=False)['Team'].apply(lambda x: Counter(x)))
grouped_general['MVP'] = participants[participants['Medal'].notna()].groupby(['Game','Sport'], as_index=False)['Medalist'].apply(lambda x:sum_MVP(list(x)))
grouped_general

0  \
Game        Sport                                                              
1896 Summer Athletics      {'Greece': 36, 'United States': 21, 'Hungary':...   
            Cycling        {'Greece': 15, 'Great Britain': 5, 'France': 7...   
            Fencing        {'France': 4, 'Greece': 9, 'Denmark': 1, 'Aust...   
            Gymnastics     {'Greece': 10, 'Germany': 66, 'Switzerland': 7...   
            Shooting       {'Greece': 46, 'Switzerland': 1, 'Denmark': 7,...   
...                                                                      ...   
2016 Summer Triathlon      {'Russia': 6, 'Spain': 6, 'Switzerland': 4, 'P...   
            Volleyball     {'Egypt': 12, 'Cameroon': 10, 'Argentina': 24,...   
            Water Polo     {'Brazil': 26, 'Japan': 12, 'Greece': 13, 'Spa...   
            Weightlifting  {'Romania': 4, 'Iraq': 1, 'Egypt': 9, 'Chile':...   
            Wrestling      {'Iran': 12, 'Uzbekistan': 8, 'Venezuela': 9, ...   

                                                                         MVP  
Game        Sport                                                             
1896 Summer Athletics      [(Robert "Bob" Garrett, 10.0), (Thomas Edmund ...  
            Cycling        [(Paul Michel Pierre Adrien Masson (-Nossam), ...  
            Fencing        [(Ioannis Georgiadis, 3.0), (Eugne Henry Grave...  
            Gymnastics     [(Hermann Otto Ludwig Weingrtner, 14.0), (Alfr...  
            Shooting       [(Ioannis Frangoudis, 6.0), (Georgios D. Orfan...  
...                                                                      ...  
2016 Summer Triathlon      [(Alistair Edward Brownlee, 3.0), (Gwen Rosema...  
            Volleyball     [(William Peixoto Arjona, 3.0), (Maurcio Borge...  
            Water Polo     [(Milan Aleksi, 3.0), (Caroline Archer "KK" Cl...  
            Weightlifting  [(Deng Wei, 3.0), (scar Albeyro Figueroa Mosqu...  
            Wrestling      [(Taha Akgl, 3.0), (Artur Aleksanyan, 3.0), (I...  

[906 rows x 2 columns]

In [318]:
grouped = grouped[grouped.MVP.notna()]


In [319]:
from collections import defaultdict

results = defaultdict(lambda: defaultdict(lambda :defaultdict(dict)))

for index, counter,medalist in grouped.itertuples():
    if medalist == np.nan:
        medalist = []
    for i, key in enumerate(index):
        if i == 0:
            nested = results[key]
        elif i == len(index) - 1:
            nested[key]['countries'] = counter
            nested[key]['medalists'] = medalist
#            nested[key] = medalist
        else:
            nested = nested[key]
            

In [320]:
results_general = defaultdict(lambda: defaultdict(lambda :defaultdict(dict)))

for index, counter,medalist in grouped_general.itertuples():
    for i, key in enumerate(index):
        if i == 0:
            nested = results_general[key]
        elif i == len(index) - 1:
            nested[key]['countries'] = counter
            nested[key]['medalists'] = medalist
        else:
            nested = nested[key]

In [321]:
def default_to_regular(d):
    if isinstance(d, defaultdict):
        d = {k: default_to_regular(v) for k, v in d.items()}
    return d

In [322]:
res_dict = default_to_regular(results)
res_gen_dict = default_to_regular(results_general)

In [323]:
with open('sports_game_details.json', 'w') as fp:
    json.dump(default_to_regular(results), fp)

with open('sports_game_general_.json', 'w') as fp:
    json.dump(default_to_regular(results_general), fp)



In [ ]:
results